In [ ]:
import os,sys
import h5py, pickle
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['figure.titlesize'] = 30
mpl.rcParams.update({"axes.grid" : True})

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
inpath='/project/s1174/fpittler/code/projectData/nucleon_sigma_term/cA211b.53.24/NJN/data_post/'

Ncfg=300

data={}

j=1
for cfg in os.listdir(inpath):
    if len(cfg) != 4:
        continue
    data[cfg]={}
    i=0
    for file in os.listdir(inpath+cfg):
        t=file.split('_')
        pat=t[2][:-3]
        if pat not in ['N']:
            continue
        with h5py.File(inpath+cfg+'/'+file) as f:
            tky=list(f.keys())[0]
            tF=f[tky]['p_p'][:]
            data[cfg][tky]=tF[:,:,0,0]
        i+=1
        print(str(j)+'/'+str(Ncfg),str(i)+'/384',end='           \r')
        # break
    j+=1
    if j>Ncfg:
        break
    # break

path='/project/s1174/lyan/code/temp/'
with open(path+'temp.pkl', 'wb') as f:
    pickle.dump(data, f)

In [ ]:
path='/project/s1174/lyan/code/temp/'
with open(path+'temp.pkl', 'rb') as f:
    data = pickle.load(f)
    
inpath='/project/s1174/fpittler/code/projectData/nucleon_sigma_term/cA211b.53.24/NJN/data_post/'
with h5py.File(inpath+'0000/Diagram0000_sx08sy06sz00st046_N.h5') as f:
    # print(f['sx08sy06sz00st46']['mvec'][:])
    print(f['sx08sy06sz00st46']['mvec'][13])
    print(f['sx08sy06sz00st46']['mvec'][14])
    print(f['sx08sy06sz00st46']['mvec'][17])
    print(f['sx08sy06sz00st46']['mvec'][26])

In [ ]:
import random

Ncfg=len(data.keys()); Nsrc=382
NsrcRef=4

t=list(range(Nsrc))
tt={}
for cfg in data.keys():
    random.shuffle(t)
    tt[cfg]=np.array(t)

fig,axs=yu.getFigAxs(1,1,12,12)
irow=0; icol=0

xs=np.arange(1,Nsrc+1)
ys=NsrcRef/xs
axs[irow,icol].plot(xs,ys**(1/2))
axs[irow,icol].set_xscale('log')
axs[irow,icol].set_xlim(0.9,Nsrc+100)
axs[irow,icol].set_xticks([1,2,4,8,16,32,64,128,256])
axs[irow,icol].get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())

NsrcList=np.array([1,2,4,8,16,32,64,128,256])

for tf in [1,5,10,15,20,25]:
    dat=np.array([[np.real(data[cfg][src][tf,13]) for src in np.array(list(data[cfg].keys()))[tt[cfg]]] for cfg in data.keys()])

    def tFunc(dat):
        t=np.cumsum(dat,axis=1)
        tt=1/np.arange(1,Nsrc+1)
        t=t*tt[None,:]
        t=(np.mean(t**2,axis=0)-np.mean(t,axis=0)**2)/(Ncfg-1)
        return t[NsrcRef-1]
    t0=tFunc(dat)
    
    def tFunc(dat):
        t=np.cumsum(dat,axis=1)
        tt=1/np.arange(1,Nsrc+1)
        t=t*tt[None,:]
        t=(np.mean(t**2,axis=0)-np.mean(t,axis=0)**2)/(Ncfg-1)
        t=t/t0
        t=[-999]+list(t)
        return np.array(t)**(1/2)
    
    (mean,cov,err)=yu.jackknife(dat,tFunc)
    tMean=mean[NsrcList]; tErr=err[NsrcList]
    xs=np.arange(500)[NsrcList]
    axs[irow,icol].errorbar(xs,tMean,tErr,fmt='s',label=tf)

axs[irow,icol].legend()
None

In [ ]:
import random

Ncfg=len(data.keys()); Nsrc=382
NsrcRef=4

t=list(range(Nsrc))
tt={}
for cfg in data.keys():
    random.shuffle(t)
    tt[cfg]=np.array(t)

fig,axs=yu.getFigAxs(1,1,12,12)
irow=0; icol=0

xs=np.arange(1,Nsrc+1)
ys=NsrcRef/xs
axs[irow,icol].plot(xs,ys**(1/2))
axs[irow,icol].set_xscale('log')
axs[irow,icol].set_xlim(0.9,Nsrc+100)
axs[irow,icol].set_xticks([1,2,4,8,16,32,64,128,256])
axs[irow,icol].get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())

NsrcList=np.array([1,2,4,8,16,32,64,128,256])

for tf in [1,24]:
    dat=np.array([[np.real(data[cfg][src][tf,13]) for src in np.array(list(data[cfg].keys()))[tt[cfg]]] for cfg in data.keys()])

    def tFunc(dat):
        t=np.cumsum(dat,axis=1)
        tt=1/np.arange(1,Nsrc+1)
        t=t*tt[None,:]
        t=(np.mean(t**2,axis=0)-np.mean(t,axis=0)**2)/(Ncfg-1)
        return t[NsrcRef-1]
    t0=tFunc(dat)
    
    def tFunc(dat):
        t=np.cumsum(dat,axis=1)
        tt=1/np.arange(1,Nsrc+1)
        t=t*tt[None,:]
        t=(np.mean(t**2,axis=0)-np.mean(t,axis=0)**2)/(Ncfg-1)
        t=t/t0
        t=[-999]+list(t)
        return np.array(t)**(1/2)
    
    (mean,cov,err)=yu.jackknife(dat,tFunc)
    tMean=mean[NsrcList]; tErr=err[NsrcList]
    xs=np.arange(500)[NsrcList]
    axs[irow,icol].errorbar(xs,tMean,tErr,fmt='s',label=tf)

axs[irow,icol].legend()
None

In [ ]:
import random

for mom in [13,14,17,26]:

    Ncfg=len(data.keys()); Nsrc=382
    NsrcRef=16

    t=list(range(Nsrc))
    tt={}
    for cfg in data.keys():
        random.shuffle(t)
        tt[cfg]=np.array(t)

    fig,axs=yu.getFigAxs(1,1,12,12)
    irow=0; icol=0

    xs=np.arange(1,Nsrc+1)
    ys=NsrcRef/xs
    axs[irow,icol].plot(xs,ys**(1/2))
    axs[irow,icol].set_xscale('log')
    axs[irow,icol].set_xlim(0.9,Nsrc+100)
    axs[irow,icol].set_xticks([1,2,4,8,16,32,64,128,256])
    axs[irow,icol].get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())

    NsrcList=np.array([1,2,4,8,16,32,64,128,256])

    for tf in [1,5,10,15,20,25]:
        dat=np.array([[np.real(data[cfg][src][tf,mom]) for src in np.array(list(data[cfg].keys()))[tt[cfg]]] for cfg in data.keys()])

        def tFunc(dat):
            t=np.cumsum(dat,axis=1)
            tt=1/np.arange(1,Nsrc+1)
            t=t*tt[None,:]
            t=(np.mean(t**2,axis=0)-np.mean(t,axis=0)**2)/(Ncfg-1)
            return t[NsrcRef-1]
        t0=tFunc(dat)
        
        def tFunc(dat):
            t=np.cumsum(dat,axis=1)
            tt=1/np.arange(1,Nsrc+1)
            t=t*tt[None,:]
            t=(np.mean(t**2,axis=0)-np.mean(t,axis=0)**2)/(Ncfg-1)
            t=t/t0
            t=[-999]+list(t)
            return np.array(t)**(1/2)
        
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        tMean=mean[NsrcList]; tErr=err[NsrcList]
        xs=np.arange(500)[NsrcList]
        axs[irow,icol].errorbar(xs,tMean,tErr,fmt='s',label=tf)

    axs[irow,icol].legend()
    None